In [ ]:
import pandas as pd #导入Pandas
df_sales = pd.read_csv('易速鲜花订单记录.csv') #载入数据
df_sales['消费日期'] = pd.to_datetime(df_sales['消费日期']) #转换日期格式
df_sales = df_sales.loc[df_sales['消费日期'] < '2021-06-01'] #只保留整月数据
df_sales = df_sales.drop_duplicates() #删除重复的数据行
df_sales = df_sales.loc[df_sales['数量'] > 0] #清洗掉数量小于等于0的数据
df_sales['总价'] = df_sales['数量'] * df_sales['单价'] #计算每单的总价


df_sales_3m = df_sales[(df_sales.消费日期 > '2020-06-01') & (df_sales.消费日期 <= '2020-08-30')] #构建仅含头三个月数据的数据集
df_sales_3m.reset_index(drop=True) #重置索引
df_user = pd.DataFrame(df_sales['用户码'].unique()) #生成以用户码为主键的结构
df_user.columns = ['用户码'] #设定字段名
df_R_value = df_sales_3m.groupby('用户码').消费日期.max().reset_index() #找到每个用户的最近消费日期，构建df_R_value对象
df_R_value.columns = ['用户码','最近购买日期'] #设定字段名
df_R_value['R值'] = (df_R_value['最近购买日期'].max() - df_R_value['最近购买日期']).dt.days #计算最新日期与上次消费日期的天数
df_user = pd.merge(df_user, df_R_value[['用户码','R值']], on='用户码') #把上次消费距最新日期的天数（R值）合并至df_user结构
df_F_value = df_sales_3m.groupby('用户码').消费日期.count().reset_index() #计算每个用户消费次数，构建df_F_value对象
df_F_value.columns = ['用户码','F值'] #设定字段名
df_user = pd.merge(df_user, df_F_value[['用户码','F值']], on='用户码') #把消费频率(F值)整合至df_user结构
df_M_value = df_sales_3m.groupby('用户码').总价.sum().reset_index() #计算每个用户三个月消费总额，构建df_M_value对象
df_M_value.columns = ['用户码','M值'] #设定字段名
df_user = pd.merge(df_user, df_M_value, on='用户码') #把消费总额整合至df_user结构

df_user_1y = df_sales.groupby('用户码')['总价'].sum().reset_index() #计算每个用户整年消费总额，构建df_user_1y对象
df_user_1y.columns = ['用户码','年度LTV'] #设定字段名
df_user_1y.head() #显示头几行数据
df_LTV = pd.merge(df_user, df_user_1y, on='用户码', how='left') #构建整体LTV训练数据集

X = df_LTV.drop(['用户码','年度LTV'],axis=1) #特征集
X.head() #显示特征集

y = df_LTV['年度LTV'] #标签集
y.head() #显示标签集

from sklearn.model_selection import train_test_split
# 先拆分训练集和其它集
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7,random_state = 0)
# 再把其它集拆分成验证集和测试集 
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,random_state = 0)

from sklearn.preprocessing import MinMaxScaler #导入归一化缩放器
scaler = MinMaxScaler() #创建归一化缩放器
X_train = scaler.fit_transform(X_train) #拟合并转换训练集数据
X_valid = scaler.transform(X_valid) #转换验证集数据
X_test = scaler.transform(X_test) #转换测试集数据

df_LTV.head()

,用户码,R值,F值,M值,年度LTV
0,15100,45,6,635.10,635.10
1,15291,35,35,1329.95,4596.51
2,14688,6,85,1472.28,4449.48
3,15311,5,715,12711.66,58218.04
4,15862,89,64,354.23,659.73


In [7]:
# 看看数据分布
print('验证集 y 的统计:')
print(y_valid.describe())

print('\n测试集 y 的统计:')
print(y_test.describe())

验证集 y 的统计:
count       54.000000
mean      2803.141111
std       6475.892841
min        115.650000
25%        404.042500
50%       1066.845000
75%       2552.987500
max      44534.300000
Name: 年度LTV, dtype: float64

测试集 y 的统计:
count       55.000000
mean      2887.565273
std       5273.554218
min         35.800000
25%        502.110000
50%       1081.750000
75%       3143.740000
max      34486.880000
Name: 年度LTV, dtype: float64


In [16]:
from sklearn.tree import DecisionTreeRegressor #导入决策树回归模型
model_dtr = DecisionTreeRegressor() #创建决策树回归模型
model_dtr_cut = DecisionTreeRegressor(max_depth=3) #创建深度为3的决策树回归模型
model_dtr.fit(X_train, y_train) #拟合决策树模型
model_dtr_cut.fit(X_train, y_train) #拟合深度为3的决策树模型
from sklearn.metrics import r2_score #导入Sklearn评估模块
print('训练集上的R平方分数-决策树: %0.4f' % r2_score(y_train, model_dtr.predict(X_train)))
print('训练集上的R平方分数-深度为3的决策树: %0.4f' % r2_score(y_train, model_dtr_cut.predict(X_train)))
print('验证集上的R平方分数-决策树: %0.4f' % r2_score(y_valid, model_dtr.predict(X_valid)))
print('验证集上的R平方分数-深度为3的决策树: %0.4f' % r2_score(y_valid, model_dtr_cut.predict(X_valid)))
print('测试集上的R平方分数-决策树: %0.4f' % r2_score(y_test, model_dtr.predict(X_test)))
print('测试集上的R平方分数-深度为3的决策树: %0.4f' % r2_score(y_test, model_dtr_cut.predict(X_test)))

训练集上的R平方分数-决策树: 1.0000
训练集上的R平方分数-深度为3的决策树: 0.8577
验证集上的R平方分数-决策树: 0.1412
验证集上的R平方分数-深度为3的决策树: 0.3284
测试集上的R平方分数-决策树: -1.6110
测试集上的R平方分数-深度为3的决策树: -0.1496


In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model_dtr, X, y, cv=5, scoring='r2')
scores_cut = cross_val_score(model_dtr_cut, X, y, cv=5, scoring='r2')

print('5次评估的 R²:', scores)
print('平均 R²:', scores.mean())
print('标准差:', scores.std())

print('5次评估的 R²:', scores)
print('平均 R²:', scores.mean())
print('标准差:', scores.std())

5次评估的 R²: [ 0.42147924 -2.52817684  0.25357147 -1.93983483 -1.86599957]
平均 R²: -1.131792105502304
标准差: 1.2226011699511583
5次评估的 R²: [ 0.42147924 -2.52817684  0.25357147 -1.93983483 -1.86599957]
平均 R²: -1.131792105502304
标准差: 1.2226011699511583
